# Exploration of Interview Data

This notebook contains helper functions that enable to explore the coded interview data further. 

In [1]:
import pandas as pd

## Data Preparation

Load the data and remove rows where the interview participant explicitly stated that they do not perceive any quality defect. However, other than in the [analysis](./../analytics/analysis.ipynb) we retain (1) incomplete and (2) unspecific data as it may be interesting for other types of investigation.

In [3]:
df = pd.read_excel('./../data/interview-data.xlsx', sheet_name='Data').fillna('na')
df = df[df['M'] == True]

c:\Users\juf\Workspace\BTH\Data Processing\rqi-pat\.venv\lib\site-packages\openpyxl\worksheet\_reader.py:312: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


In [5]:
# list the variables containing codes, i.e., remove all variables which contain supplementary information or verbatim mentions
allvars = [ 
    'ID',
    'Quality Factor 1', 'Entity-Fact 1', 'Quality Factor 2', 'Entity-Fact 2',
    'Context Factor 1', 'Context Factor 2', 'Context Factor 3',
    'Activity 1', 'Attribute 1', 'Impact 1', 'Activity 2', 'Attribute 2', 'Impact 2'
]

## Helper functions

In the following code blocks, we define helper functions which generate query strings for the `pandas.DataFrame.query()` method. By providing for example the `query_qf()` method with a quality factor, it constructs a query that searches for all rows in which either the column `Quality Factor 1` or `Quality Factor 2` equal the provided value.

In [6]:
def query_qf(qualityfactor: str, entityfact: str = "") -> str : 
    queries: [str] = []
    for idx in ["1", "2"]:
        query: str = f'`Quality Factor {idx}`=="{qualityfactor}"'
        if entityfact:
            query += f' & `Entity-Fact {idx}`=="{entityfact}"'
        queries.append(query)

    return '(' + ' | '.join(queries) + ')'

In [7]:
def query_context(contextfactors: [str]) -> str :
    col_ctx: [str] = ['Context Factor 1', 'Context Factor 2', 'Context Factor 3']

    queries = [f'`{column}` in {contextfactors}' for column in col_ctx]

    return '(' + ' | '.join(queries) + ')'

In [8]:
def query_activity(activity: str, attribute: str = "", impact: int = None) -> str:
    queries: [str] = []
    for idx in ["1", "2"]:
        query = f'`Activity {idx}`=="{activity}"'
        if attribute:
            query += f' & `Attribute {idx}`=="{attribute}"'
            if impact:
                query += f' & `Impact {idx}`=={impact}'
        queries.append(query)

    return '(' + ' | '.join(queries) + ')'

## Exploration

With the helper methods, query strings can be generated that allow to filter the existing data for specific attributes.

The first query retrieves all statements in which the quality factor `orientation` impacts the activity of `Understanding`.

In [9]:
qqf = query_qf(qualityfactor='orientation')
qac = query_activity(activity='Understanding')
df.query(f'{qqf} & {qac}')[allvars]

,ID,Quality Factor 1,Entity-Fact 1,Quality Factor 2,Entity-Fact 2,Context Factor 1,Context Factor 2,Context Factor 3,Activity 1,Attribute 1,Impact 1,Activity 2,Attribute 2,Impact 2
0,1,orientation,solution,na,na,na,na,na,Understanding,Uniqueness,-3.0,na,na,na
1,2,orientation,solution,na,na,Involvement,na,na,Understanding,Uniqueness,0.0,na,na,na
12,13,orientation,solution,na,na,na,na,na,Understanding,Unspecific,-3.0,na,na,na
54,55,orientation,solution,na,na,na,na,na,Understanding,Uniqueness,-2.0,na,na,na
55,56,orientation,solution,na,na,Peer Review,na,na,Understanding,Uniqueness,0.0,na,na,na
103,104,orientation,solution,na,na,na,na,na,Understanding,Unspecific,1.0,Translating,Unspecific,2.0


The second query retrieves all statements in which the quality factor `orientation` with the value `solution` and the context factor `Involvement` impact the attribute `Uniqueness` of the activity `Understanding`.

In [10]:
qqf = query_qf(qualityfactor='orientation', entityfact='solution')
qct = query_context(contextfactors=['Involvement'])
qac = query_activity(activity='Understanding', attribute='Uniqueness')
df.query(f'{qqf} & {qct} & {qac}')[allvars]

,ID,Quality Factor 1,Entity-Fact 1,Quality Factor 2,Entity-Fact 2,Context Factor 1,Context Factor 2,Context Factor 3,Activity 1,Attribute 1,Impact 1,Activity 2,Attribute 2,Impact 2
1,2,orientation,solution,na,na,Involvement,na,na,Understanding,Uniqueness,0.0,na,na,na
